In [1]:
from dash import Dash, html, dcc, Input, Output, callback, jupyter_dash
jupyter_dash.default_mode="external"
import pandas as pd
import plotly.express as px
import numpy as np

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

def drop_single_val_indexes(df):
    lvls_to_drop = [lvl_idx for lvl_idx, lvl_name in enumerate(df.index.names)\
                    if len(df.index.get_level_values(lvl_idx).unique()) == 1]
    return df.droplevel(lvls_to_drop)

In [2]:
map = pd.read_hdf('../data/L5Hay/spatial_profiles.hdf')
map = drop_single_val_indexes(map)
# map = map.loc['L5_Hay2011_cell2_vertical_shaft_10higherapicsod']
map = map.loc['L5_Hay2011_cell1']
traces = pd.read_hdf('../data/L5Hay/NEURONvars.hdf')
traces['model'] = traces['fname'].apply(lambda x:x.split('_')[1].split('/')[1])
# traces = traces.loc[traces.model == 'L5Haycell2vs10sodapd']
traces = traces.loc[traces.model == 'L5Haycell1']
traces = drop_single_val_indexes(traces)
traces['intensity_mWPERmm2'] = traces['fname'].apply(lambda x:float(x.split('__')[-1][:-4]))
traces['light_power'] = traces['intensity_mWPERmm2'] * 0.1*0.1*np.pi /1000
traces = traces.set_index(['light_power', 'radius', 'angle'])
# round lp to avoid floating point related key errors
def round_lp(df, set_index):
    df = df.reset_index()
    df.light_power = df.light_power.round(10)
    return df.set_index(set_index)
map = round_lp(map, set_index=['light_power', 'radius [um]', 'angle [rad]'])
traces = round_lp(traces, set_index=['light_power', 'radius', 'angle'])

# reduce to relevant responses:
#map = map.loc[1.5e-4:]
#map = map.loc[:,:200,:]
#traces = traces.loc[1.5e-4:]

In [3]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def cvrt_x_y_in_r_a(x,y, df_r_a_val):
    radii = df_r_a_val.reset_index()['radius [um]'].unique()
    angles = df_r_a_val.reset_index()['angle [rad]'].unique()
    radius = find_nearest(radii, np.sqrt(x**2+y**2))
    angle = find_nearest(angles, ((np.arctan2(y, x) + 2*np.pi) % (2*np.pi)))                    
    return radius, angle
    
def cvrt_polar_to_cartesian(df_r_a_val):
    # sampling distance
    d_smpls = 10
    radii = df_r_a_val.reset_index()['radius [um]'].unique()
    x = np.arange(-radii.max(), radii.max()+d_smpls, d_smpls)
    xx, yy = np.meshgrid(x,x)
    cartesian_df = []
    for x, y in zip(xx.flatten(), yy.flatten()):
        radius, angle = cvrt_x_y_in_r_a(x,y, df_r_a_val)
        value = df_r_a_val.loc[radius,angle]['AP_count']
        cartesian_df.append(dict(x=x, y=y, value=value))
    return pd.DataFrame(cartesian_df)

In [4]:
lps = map.reset_index().light_power.unique()
app = Dash(__name__, external_stylesheets=external_stylesheets)

mapfigs = [px.imshow(cvrt_polar_to_cartesian(
    map.loc[lp]).pivot(columns='x', index='y', values='value'), 
    color_continuous_scale='Blues', 
    zmin=0, zmax=15,
    title=str(lp/(0.1*0.1)),
    origin='lower') for lp in lps[0:5]]
vtracefigs = [px.line(traces.loc[lp, 0, 0][['time [ms]', 'V_soma(0.5)']].reset_index(), x='time [ms]', y='V_soma(0.5)')\
           for lp in lps[0:5]]
    
app.layout = html.Div([
    html.Div([
        html.Label('map0'),
        dcc.Graph(
            id='map0',
            figure=mapfigs[0],
        ),
        dcc.Dropdown(
            lps,
            lps[0],
            id='dd-lp0'
        ),
        html.Label('vtrace0'),
        dcc.Graph(
            id='vtrace0',
            figure=vtracefigs[0],
        ),
    ], style={'width': '19%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        html.Label('map1'),
        dcc.Graph(
            id='map1',
            figure=mapfigs[1],
        ),
        dcc.Dropdown(
            lps,
            lps[1],
            id='dd-lp1'
        ),
        html.Label('vtrace1'),
        dcc.Graph(
            id='vtrace1',
            figure=vtracefigs[1],
        ),
    ], style={'width': '19%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        html.Label('map2'),
        dcc.Graph(
            id='map2',
            figure=mapfigs[2],
        ),
        dcc.Dropdown(
            lps,
            lps[2],
            id='dd-lp2'
        ),
        html.Label('vtrace2'),
        dcc.Graph(
            id='vtrace2',
            figure=vtracefigs[2],
        ),
    ], style={'width': '19%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        html.Label('map3'),
        dcc.Graph(
            id='map3',
            figure=mapfigs[3],
        ),
        dcc.Dropdown(
            lps,
            lps[3],
            id='dd-lp3'
        ),
        html.Label('vtrace3'),
        dcc.Graph(
            id='vtrace3',
            figure=vtracefigs[3],
        ),
    ], style={'width': '19%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        html.Label('map4'),
        dcc.Graph(
            id='map4',
            figure=mapfigs[4],
        ),
        dcc.Dropdown(
            lps,
            lps[4],
            id='dd-lp4'
        ),
        html.Label('vtrace4'),
        dcc.Graph(
            id='vtrace4',
            figure=vtracefigs[4],
        ),
    ], style={'width': '19%', 'display': 'inline-block', 'padding': '0 20'}),
])

def create_map(map, lp):
    mapfig = px.imshow(cvrt_polar_to_cartesian(
        map.loc[lp]).pivot(columns='x', index='y', values='value'), 
        color_continuous_scale='Blues', 
        zmin=0, zmax=15,
        title=str(lp/(0.1*0.1)),
        origin='lower')
    return mapfig

@callback(
    Output('map0', 'figure'),
    Input('dd-lp0', 'value'),
)
def update_map0(lp):
    return create_map(map, lp)

@callback(
    Output('map1', 'figure'),
    Input('dd-lp1', 'value'),
)
def update_map1(lp):
    return create_map(map, lp)

@callback(
    Output('map2', 'figure'),
    Input('dd-lp2', 'value'),
)
def update_map2(lp):
    return create_map(map, lp)

@callback(
    Output('map3', 'figure'),
    Input('dd-lp3', 'value'),
)
def update_map3(lp):
    return create_map(map, lp)

@callback(
    Output('map4', 'figure'),
    Input('dd-lp4', 'value'),
)
def update_map4(lp):
    return create_map(map, lp)

def create_v_trace(lp, hoverData, clickData):
    hover = True
    click = True
    radiush, angleh = [0,0]
    radiusc, anglec = [0,0]
    try:
        radiush, angleh = cvrt_x_y_in_r_a(
            x=hoverData['points'][0]['x'],
            y=hoverData['points'][0]['y'], 
            df_r_a_val=map
        )
    except TypeError:
        hover = False
    try:
        radiusc, anglec = cvrt_x_y_in_r_a(
            x=clickData['points'][0]['x'],
            y=clickData['points'][0]['y'], 
            df_r_a_val=map
        )
    except TypeError:
        click = False
    if hover and click:
        hovertrace = traces.loc[lp, radiush, angleh][['time [ms]', 'V_soma(0.5)']]
        hovertrace['radius, angle']=''.join([str(radiush),'um, ',str(np.round(angleh/np.pi,1)),'pi'])
        clicktrace = traces.loc[lp, radiusc, anglec][['time [ms]', 'V_soma(0.5)']]
        clicktrace['radius, angle']=''.join([str(radiusc),'um, ',str(np.round(anglec/np.pi,1)),'pi'])
        trace = pd.concat([hovertrace,clicktrace])
        return px.line(trace.reset_index(), x='time [ms]', y='V_soma(0.5)', color='radius, angle')
    elif hover:    
        hovertrace = traces.loc[lp, radiush, angleh][['time [ms]', 'V_soma(0.5)']]
        hovertrace['radius, angle']=''.join([str(radiush),'um, ',str(np.round(angleh/np.pi,1)),'pi'])
        clicktrace = hovertrace.copy()
        clicktrace['V_soma(0.5)'] = np.nan
        clicktrace['radius, angle']=''.join([str(radiusc),'um, ',str(np.round(anglec/np.pi,1)),'pi'])
        trace = pd.concat([hovertrace,clicktrace])
        return px.line(trace.reset_index(), x='time [ms]', y='V_soma(0.5)', color='radius, angle')
    elif click:    
        clicktrace = traces.loc[lp, radiusc, anglec][['time [ms]', 'V_soma(0.5)']]
        clicktrace['radius, angle']=''.join([str(radiusc),'um, ',str(np.round(anglec/np.pi,1)),'pi'])
        hovertrace = clicktrace.copy()
        hovertrace['V_soma(0.5)'] = np.nan
        hovertrace['radius, angle']=''.join([str(radiusc),'um, ',str(np.round(anglec/np.pi,1)),'pi'])
        trace = pd.concat([hovertrace,clicktrace])
        return px.line(trace.reset_index(), x='time [ms]', y='V_soma(0.5)', color='radius, angle')
    else:    
        return px.line(x=[0,1],y=[0,0])

@callback(
    Output('vtrace0', 'figure'),
    Input('dd-lp0', 'value'),
    Input('map0', 'hoverData'),
    Input('map0', 'clickData'),
)
def update_vtrace(lp, hoverData, clickData):
    return create_v_trace(lp, hoverData, clickData)
    
@callback(
    Output('vtrace1', 'figure'),
    Input('dd-lp1', 'value'),
    Input('map1', 'hoverData'),
    Input('map1', 'clickData'),
)
def update_vtrace(lp, hoverData, clickData):
    return create_v_trace(lp, hoverData, clickData)
    
@callback(
    Output('vtrace2', 'figure'),
    Input('dd-lp2', 'value'),
    Input('map2', 'hoverData'),
    Input('map2', 'clickData'),
)
def update_vtrace(lp, hoverData, clickData):
    return create_v_trace(lp, hoverData, clickData)
    
@callback(
    Output('vtrace3', 'figure'),
    Input('dd-lp3', 'value'),
    Input('map3', 'hoverData'),
    Input('map3', 'clickData'),
)
def update_vtrace(lp, hoverData, clickData):
    return create_v_trace(lp, hoverData, clickData)
    
@callback(
    Output('vtrace4', 'figure'),
    Input('dd-lp4', 'value'),
    Input('map4', 'hoverData'),
    Input('map4', 'clickData'),
)
def update_vtrace(lp, hoverData, clickData):
    return create_v_trace(lp, hoverData, clickData)
    
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>